In [ ]:
import sys
!{sys.executable} -m pip install pymongo --user

In [ ]:
from pymongo import MongoClient
from pymongo import ReplaceOne
from pymongo.errors import BulkWriteError
from pymongo.errors import OperationFailure
import uuid
import os
import random
import string
import time
import logging

def retry(func, *func_args, **kwargs):
    retry_count = kwargs.pop("retry_count", 20)
    delay = kwargs.pop("delay", 0.1)
    attempt = 0
    
    while (attempt <= retry_count):
        logging.debug('Attempt: %s', str(attempt))
        try:
            return func(*func_args, **kwargs)
        except OperationFailure as failure:
            if (attempt < retry_count and failure.code == 16500):
                logging.debug("waiting for %s seconds before retyring again", str(delay))
                time.sleep(delay) 
                attempt += 1
            else:
                raise

logging.getLogger().propagate = True
#logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger().setLevel(logging.INFO)

uri = 'mongodb://' + \
    os.environ['DB_ACCOUNT_NAME'] + \
    ':' + \
    os.environ['COSMOS_KEY'] + \
    '@' + \
    os.environ['DB_ACCOUNT_NAME'] + \
    '.documents.azure.com:10255/?ssl=true&replicaSet=globaldb'

client = MongoClient(uri)
database = client.get_database(name='testdb')
collection = database.get_collection(name='source')

for x in range (0, 5):
    requests = []

    for i in range(0, 16):
        rowId = str(uuid.uuid4())
        payload = str(uuid.uuid4())

        requests.append(ReplaceOne(
            filter = { 'id': rowId },
            replacement = {
                'id': rowId,
                'payload': payload
                },
            upsert = True))

    startTime = time.perf_counter()

    result = retry(collection.bulk_write,
        requests = requests,
        ordered = False,
        bypass_document_validation = True,
        retry_count = 20,
        delay = 0.05)
    
    endTime = time.perf_counter()
    duration =  endTime - startTime
    
    logging.info('')
    logging.info('Iteration: %s', str(x))    
    logging.info('Duration: %s', str(duration))

    logging.debug('StartTime: %s', str(startTime))
    logging.debug('EndTime: %s', str(endTime))
    logging.debug('Acknowledged: %s', str(result.acknowledged))

    if result.acknowledged:
        logging.debug('Deleted: %s', str(result.deleted_count))
        logging.debug('Inserted: %s', str(result.inserted_count))
        logging.debug('Matched: %s', str(result.matched_count))
        logging.debug('Modified: %s', str(result.modified_count))
        logging.debug('Upserted: %s', str(result.upserted_count))

print("Finished")